In [3]:
import numpy as np
import pandas as pd
def extract_time(data, time_column):
    '''This function assumes that time is in a datetime format. 
        It takes this datetime column and extracts the following in a sin_cos format:
        - Month
        - Day of week
        - Hour
    '''
    _data = data.copy()
    # Extracting month, day of week, and hour from the datetime column
    _data['Month_sin'] = np.sin(2 * np.pi * data[time_column].dt.month / 12)
    _data['Month_cos'] = np.cos(2 * np.pi * data[time_column].dt.month / 12)
    
    _data['DayOfWeek_sin'] = np.sin(2 * np.pi * data[time_column].dt.dayofweek / 7)
    _data['DayOfWeek_cos'] = np.cos(2 * np.pi * data[time_column].dt.dayofweek / 7)
    
    _data['Hour_sin'] = np.sin(2 * np.pi * data[time_column].dt.hour / 24)
    _data['Hour_cos'] = np.cos(2 * np.pi * data[time_column].dt.hour / 24)
    
    # Dropping the original time column
    # data.drop(columns=[time_column], inplace=True) # Uncomment if you want to drop the original time column
    
    return _data

def lag_agg_day(data, time_column, groupby_variable, variable, n_lags, agg_method, new_col_name):
    '''Lags and aggregates data based on the day without considering day of the week.
    It applies aggregation methods like mean or sum based on the agg_method parameter.'''
    _data = data.copy()
    _data.set_index(time_column, inplace=True)
    if agg_method == 'mean':
        _data[new_col_name] = _data.groupby([_data.index.hour, groupby_variable], sort=False)[variable].transform(lambda x: x.shift(1).rolling(n_lags).mean())
    elif agg_method =='sum':
        _data[new_col_name] = _data.groupby([_data.index.hour, groupby_variable], sort=False)[variable].transform(lambda x: x.shift(1).rolling(n_lags).sum())
    _data.reset_index(inplace=True)
    return _data

def lag_agg_dayofweek(data, time_column, groupby_variable, variable, n_lags, agg_method, new_col_name):
    '''Lags and aggregates data based on the day of the week.
    It applies aggregation methods like mean or sum based on the agg_method parameter.'''
    _data = data.copy()
    _data.set_index(time_column, inplace=True)
    if agg_method == 'mean':
        _data[new_col_name] = _data.groupby([_data.index.hour, _data.index.dayofweek, groupby_variable], sort=False)[variable].transform(lambda x: x.shift(1).rolling(n_lags).mean())
    elif agg_method =='sum':
        _data[new_col_name] = _data.groupby([_data.index.hour, _data.index.dayofweek, groupby_variable], sort=False)[variable].transform(lambda x: x.shift(1).rolling(n_lags).sum())
    _data.reset_index(inplace=True)
    return _data

def lag_agg(data, time_column, groupby_variable, variable, n_lags, agg_method, new_col_name):
    '''Lags and aggregates data based on a specified variable.
    It applies aggregation methods like mean or sum based on the agg_method parameter.'''
    _data = data.copy()
    _data.set_index(time_column, inplace=True)
    if agg_method == 'mean':
        _data[new_col_name] = _data.groupby([groupby_variable], sort=False)[variable].transform(lambda x: x.rolling(n_lags).mean())
    elif agg_method =='sum':
        _data[new_col_name] = _data.groupby([groupby_variable], sort=False)[variable].transform(lambda x: x.rolling(n_lags).sum())
    _data.reset_index(inplace=True)
    return _data

def hour_agg(data, groupby_columns, time_column, value_column):
    """
    Perform hourly aggregation on the specified DataFrame.

    Parameters:
    - data: DataFrame to be aggregated.
    - groupby_columns: List of columns to group by.
    - time_column: Name of the time column.
    - value_column: Name of the column to aggregate.

    Returns:
    - Aggregated DataFrame.
    """
    return (
        data.groupby([*groupby_columns, data[time_column].dt.round('H')], sort=False)
            .agg({value_column: 'sum'})
            .reset_index()
    )

def _fill_missing_dates(df: pd.DataFrame, min_date: pd.Timestamp, max_date: pd.Timestamp) -> pd.DataFrame:
    """Fill missing dates in the time series between the minimum and maximum dates and set their
    values to NaN.

    :param df: Time series sales data for a specific country-brand.
    :param min_date: Minimum date to be considered.
    :param max_date: Maximum date to be considered.
    :return: Complete time series for a specific country-type.
    """

    df['Time'] = pd.to_datetime(df['Time'])
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='H')
    complete_df = (
        pd.DataFrame({'Time': complete_date_range})
        .merge(df[['CountryID']].drop_duplicates(), how='cross')
    )
    result_df = complete_df.merge(df, on=['Time', 'CountryID'], how='left')

    return result_df

In [4]:
pd.set_option('display.max_colwidth', None) 

load_data = pd.read_csv('C:\\Users\\RULLFLL1\\SE_competition\\SE_Competition\\data\\master_load.csv').drop('Unnamed: 0', axis=1)
gen_data = pd.read_csv('C:\\Users\\RULLFLL1\\SE_competition\\SE_Competition\\data\\master_gen.csv').drop('Unnamed: 0', axis=1)

# datetime objects
load_data['Time'] = pd.to_datetime(load_data.Time)
gen_data['Time'] = pd.to_datetime(gen_data.Time)

# drop column
gen_data.drop(['AreaID', 'UnitName'], axis=1, inplace=True)
load_data.drop(['AreaID', 'PsrType', 'UnitName'], axis=1, inplace=True)

In [5]:
all_data = {'load_data': load_data, 'gen_data': gen_data}

for name, df in all_data.items():
    if 'PsrType' in df.columns:
        all_data[name] = hour_agg(df, ['CountryID', 'PsrType'], 'Time', 'quantity')
    else:
        all_data[name] = hour_agg(df, ['CountryID'], 'Time', 'quantity')

load_data = all_data['load_data']
gen_data = all_data['gen_data']

In [11]:
# Load Data
datetime_series = pd.date_range(start=load_data.Time.min(), end=load_data.Time.max(), freq='H')

# Creating a range from 0 to 9
numbers_range = load_data.CountryID.unique()

# Creating a multi-index from cartesian product of both ranges
index = pd.MultiIndex.from_product([datetime_series, numbers_range], names=['Time', 'CountryID'])

# Creating a DataFrame with the multi-index
df_index = pd.DataFrame(index=index).reset_index()

load_data_full = df_index.merge(load_data, how='left')

In [12]:
# Gen Data
datetime_series = pd.date_range(start=gen_data.Time.min(), end=gen_data.Time.max(), freq='H')
numbers_range = gen_data.CountryID.unique()
psr_vals = gen_data.PsrType.unique()

# Creating a multi-index from cartesian product of both ranges
index = pd.MultiIndex.from_product([datetime_series, numbers_range, psr_vals], names=['Time', 'CountryID', 'PsrType'])

# Creating a DataFrame with the multi-index
df_index = pd.DataFrame(index=index).reset_index()

gen_data_full = df_index.merge(gen_data, how='left')

In [13]:
# note that I'm simply filling missing values as zero. May need to reconsider
gen_data_full = gen_data_full.pivot_table(index = ['Time','CountryID'], columns= ['PsrType'], values='quantity')

In [16]:
# Merging load and gen
data = load_data_full.rename({'quantity':'load'}, axis=1).set_index(['CountryID','Time']).merge(gen_data_full, left_index=True, right_index=True)
data = data.reset_index()
data.head()

,CountryID,Time,load,B01,B02,B03,B04,B05,B06,B07,...,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20
0,2,2022-01-01,42196.0,4333.0,3565.0,NaN,2634.0,2053.0,279.0,NaN,...,1424.0,119.0,NaN,3351.0,126.0,0.0,846.0,5688.0,24850.0,333.0
1,3,2022-01-01,3314.0,468.0,NaN,NaN,215.0,331.0,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,169.0,1875.0,1308.0,NaN
2,5,2022-01-01,4254.0,132.0,328.0,NaN,504.0,0.0,0.0,NaN,...,11.0,9.0,NaN,1950.0,7.0,0.0,15.0,NaN,126.0,69.0
3,6,2022-01-01,21121.0,677.0,NaN,517.0,6234.0,1465.0,202.0,NaN,...,2115.0,242.0,NaN,NaN,NaN,0.0,36.0,NaN,2024.0,1365.0
4,8,2022-01-01,10326.0,21.0,NaN,NaN,2019.0,2302.0,NaN,NaN,...,0.0,NaN,NaN,485.0,NaN,1.0,410.0,2016.0,1516.0,3568.0


In [17]:
def _fill_missing_dates(df: pd.DataFrame, min_date: pd.Timestamp, max_date: pd.Timestamp) -> pd.DataFrame:
    """Fill missing dates in the time series between the minimum and maximum dates and set their
    values to NaN.

    :param df: Time series sales data for a specific country-brand.
    :param min_date: Minimum date to be considered.
    :param max_date: Maximum date to be considered.
    :return: Complete time series for a specific country-type.
    """

    df['Time'] = pd.to_datetime(df['Time'])
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='H')
    complete_df = (
        pd.DataFrame({'Time': complete_date_range})
        .merge(df[['CountryID']].drop_duplicates(), how='cross')
    )
    result_df = complete_df.merge(df, on=['Time', 'CountryID'], how='left')

    return result_df

In [36]:
data_clean = _fill_missing_dates(data,min_date=data['Time'].min(),max_date=data['Time'].max())
data_clean

,Time,CountryID,load,B01,B02,B03,B04,B05,B06,B07,...,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20
0,2022-01-01,2,42196.0,4333.0,3565.0,NaN,2634.0,2053.0,279.0,NaN,...,1424.0,119.0,NaN,3351.0,126.0,0.0,846.0,5688.0,24850.0,333.0
1,2022-01-01,3,3314.0,468.0,NaN,NaN,215.0,331.0,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,169.0,1875.0,1308.0,NaN
2,2022-01-01,5,4254.0,132.0,328.0,NaN,504.0,0.0,0.0,NaN,...,11.0,9.0,NaN,1950.0,7.0,0.0,15.0,NaN,126.0,69.0
3,2022-01-01,6,21121.0,677.0,NaN,517.0,6234.0,1465.0,202.0,NaN,...,2115.0,242.0,NaN,NaN,NaN,0.0,36.0,NaN,2024.0,1365.0
4,2022-01-01,8,10326.0,21.0,NaN,NaN,2019.0,2302.0,NaN,NaN,...,0.0,NaN,NaN,485.0,NaN,1.0,410.0,2016.0,1516.0,3568.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78844,2023-01-01,8,29551.0,135.0,NaN,NaN,3412.0,3789.0,NaN,NaN,...,0.0,NaN,NaN,1458.0,NaN,6.0,1223.0,3633.0,5343.0,14844.0
78845,2023-01-01,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78846,2023-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78847,2023-01-01,0,39824.0,336.0,0.0,0.0,5732.0,644.0,80.0,0.0,...,1872.0,4340.0,0.0,12928.0,160.0,40.0,464.0,0.0,12028.0,24.0


In [37]:
def calculate_green_energy(df):
    green_energy = ['B01', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19']
    df['green_energy'] = df[green_energy].sum(axis=1, skipna=True)
    df = df[['Time', 'CountryID', 'load', 'green_energy']]
    return df

In [38]:
data_clean_2 = data_clean.groupby(['Time', 'CountryID']).apply(calculate_green_energy)

data_clean_2 = data_clean_2.reset_index(drop=True)

In [39]:
def pivot_and_flatten(df, index_col, country_col, value_cols, aggfunc='first'):
    """
    Pivot the DataFrame from long to wide, flatten multi-level columns, and reset the index.

    Parameters:
    - df: Input DataFrame.
    - index_col: Column to be used as the index in the wide DataFrame.
    - country_col: Column to be used as columns in the wide DataFrame.
    - value_cols: List of columns to be used as values in the wide DataFrame.
    - aggfunc: Aggregation function for pivot_table.

    Returns:
    - Wide DataFrame with flattened columns and reset index.
    """
    # Pivot the DataFrame from long to wide
    wide_df = df.pivot_table(index=index_col, columns=country_col, values=value_cols, aggfunc=aggfunc)

    # Flatten the multi-level columns
    wide_df.columns = [f'{col}_{country}' for col, country in wide_df.columns]

    # Resetting the index
    wide_df = wide_df.reset_index()

    return wide_df

In [40]:
data_clean_wide = pivot_and_flatten(data_clean_2, index_col='Time', country_col='CountryID', value_cols=['green_energy', 'load'])

In [41]:
data_clean_wide

,Time,green_energy_0,green_energy_1,green_energy_2,green_energy_3,green_energy_4,green_energy_5,green_energy_6,green_energy_7,green_energy_8,load_0,load_1,load_2,load_3,load_4,load_5,load_6,load_7,load_8
0,2022-01-01 00:00:00,17603.0,0.0,43598.0,3821.0,16902.0,2250.0,5837.0,4641.0,4449.0,20827.0,NaN,42196.0,3314.0,15331.0,4254.0,21121.0,14438.0,10326.0
1,2022-01-01 01:00:00,17184.0,0.0,174116.0,3774.0,16946.0,9228.0,5781.0,4491.0,16327.0,19530.0,NaN,165125.0,3218.0,15331.0,16457.0,19756.0,13935.0,40706.0
2,2022-01-01 02:00:00,17497.0,0.0,168446.0,3478.0,16875.0,9374.0,6264.0,4436.0,14713.0,18383.0,NaN,160415.0,3126.0,15270.0,15426.0,18685.0,13579.0,39465.0
3,2022-01-01 03:00:00,17712.0,0.0,164624.0,3212.0,16349.0,9406.0,6442.0,4568.0,14893.0,17680.0,NaN,158035.0,3080.0,15150.0,14781.0,18124.0,13397.0,38923.0
4,2022-01-01 04:00:00,17605.0,0.0,158479.0,3021.0,16609.0,9138.0,5797.0,4559.0,14878.0,17396.0,NaN,157016.0,3044.0,15387.0,14630.0,18400.0,13364.0,38211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2022-12-31 20:00:00,115688.0,824.0,192166.0,1902.0,15976.0,8740.0,4242.0,6959.0,20820.0,128248.0,NaN,184106.0,3713.0,14440.0,17488.0,24809.0,15211.0,45646.0
8757,2022-12-31 21:00:00,65924.0,830.0,193062.0,1730.0,15383.0,8773.0,3719.0,6910.0,20854.0,69520.0,NaN,175401.0,3579.0,14201.0,17075.0,23169.0,14641.0,43425.0
8758,2022-12-31 22:00:00,106916.0,676.0,191073.0,1920.0,14944.0,8689.0,3685.0,7054.0,20756.0,106396.0,NaN,168830.0,3520.0,13713.0,16717.0,21857.0,13977.0,41766.0
8759,2022-12-31 23:00:00,60620.0,489.0,178308.0,2362.0,14605.0,8618.0,3466.0,5946.0,19525.0,60520.0,NaN,161541.0,3440.0,13772.0,16261.0,20555.0,13272.0,40415.0


In [42]:
data_clean_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Time            8761 non-null   datetime64[ns]
 1   green_energy_0  8761 non-null   float64       
 2   green_energy_1  8761 non-null   float64       
 3   green_energy_2  8761 non-null   float64       
 4   green_energy_3  8761 non-null   float64       
 5   green_energy_4  8761 non-null   float64       
 6   green_energy_5  8761 non-null   float64       
 7   green_energy_6  8761 non-null   float64       
 8   green_energy_7  8761 non-null   float64       
 9   green_energy_8  8761 non-null   float64       
 10  load_0          8761 non-null   float64       
 11  load_1          1572 non-null   float64       
 12  load_2          8761 non-null   float64       
 13  load_3          8760 non-null   float64       
 14  load_4          8759 non-null   float64       
 15  load

In [43]:
def impute_nans(df):
    """
    Impute NaN values in a DataFrame with specific logic:
    - If all rows in a column are missing, set the value to 0.
    - Otherwise, impute the NaN with the mean between the previous and the following value.

    Parameters:
    - df: Input DataFrame.

    Returns:
    - DataFrame with NaN values imputed based on the specified logic.
    """
    for col in df.columns:
        # Check if all rows in the column are missing
        if df[col].isnull().all():
            # Set the value to 0 if all rows are missing
            df[col] = 0
        elif df[col].dtype == 'datetime64[ns]':
            # Impute NaN with the mean of the datetime values
            df[col] = df[col].fillna(df[col].mean())
        else:
            # Impute NaN with the mean between the previous and the following value
            df[col] = df[col].fillna((df[col].shift() + df[col].shift(-1)) / 2)

    return df

data_clean_wide_imputed = impute_nans(data_clean_wide)
